# 欢迎语
更新日志

## <i class="fa fa-repeat"></i> 切换pip软件源

In [ ]:
import os
import subprocess

def check_pip_list():
    output = !pip config list
    print(output)
    if "global.index-url='https://pypi.tuna.tsinghua.edu.cn/simple'" in output and "global.trusted-host='pypi.tuna.tsinghua.edu.cn'" in output:
        return True
    else:
        return False
if not check_pip_list():
    # 查找pip.conf文件
    pip_conf_file = os.popen("find / -name 'pip.conf'").read().strip()
    # 直接覆盖pip.conf文件
    with open(pip_conf_file, 'w') as f:
        f.write("[global]\nindex-url = https://pypi.tuna.tsinghua.edu.cn/simple\ntrusted-host = pypi.tuna.tsinghua.edu.cn")
        # f.write("")
    # 验证修改是否成功
    if check_pip_list():
        print("pip软件源切换成功！")
    else:
        print("pip软件源切换失败，请检查脚本是否正确运行。")
else:
    print("pip软件源本来就是清华源，无需切换")

In [ ]:
new_channels = [
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/msys2/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/conda-forge/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/peterjc123/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/pytorch/"
]

current_channels = !conda config --show channels
current_channels = ('').join(current_channels).split()

# print(current_channels)

for channel in new_channels:
    if channel in current_channels:
        print(f"此channel已经在列表中：{channel}")
    else:
        !conda config --add channels $channel
        
!conda config --set show_channel_urls yes

##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 判断当前脚本目录是否存在`.git`文件夹
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
# print(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)
# 获取当前环境名称与当前环境content路径
from func.env import detect_environment
env_cb = detect_environment()
env_name = env_cb['env_name']
content_path = env_cb['content_path']

# 装载nbfunc中的hub涉及的所有ipynb函数
%cd $repo_root_dir
%run ./nbfunc/hub.ipynb

initGit()

## <i class="fa fa-arrow-down"></i> 安装必要依赖

In [ ]:
import subprocess

libraries_to_install = [

]

tools_to_install = [
    "iputils-ping",
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install $library --user
    else:
        print(f"【 {library} 】已经安装，跳过安装")
        
# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool

# 基础配置

In [ ]:
#需要安装ipywidgets，此镜像已经安装了对应的依赖包
# !conda install -c conda-forge jupyterlab 
# !conda install -c conda-forge ipywidgets
# !conda install -c conda-forge ipython
# !conda install -c conda-forge ipympl

# 实现显示/隐藏代码块的步骤：
# 安装jupyter_contrib_nbextentions
# （Ref: https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html#install-the-python-package）
# conda安装命令：conda install -c conda-forge jupyter_contrib_nbextensions
# python安装命令：pip install jupyter_contrib_nbextensions

# 安装jupyter_nbextentions_contributor，以便在Jupyter Notebook页面设置nbextentions
# （Ref: https://github.com/Jupyter-contrib/jupyter_nbextensions_configurator）
# conda安装命令：conda install -c conda-forge jupyter_nbextensions_configurator
# python安装命令：pip install jupyter_nbextensions_configurator

# !apt-get update && apt install net-tools

## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
# from func.env import setProxy
# try:
#     proxy,region
# except NameError:
#     sp = setProxy()
#     proxy = sp['proxy']
#     region = sp['region']
%cd $repo_root_dir
%run ./nbfunc/hub.ipynb
selProxy(True,'自供')
print(proxy)
print(proxyURL)

## 设置Git使用代理访问GitHub

In [ ]:
def set_git_proxy(proxyURL):
    # 设置Git在使用HTTP协议访问GitHub时使用代理服务器
    !git config --global http.https://github.com.proxy $proxyURL

    # 设置Git在使用HTTPS协议访问GitHub时使用代理服务器
    !git config --global https.https://github.com.proxy $proxyURL

    # 验证是否设置成功
    result = !git config --list
    # print(result)
    result = [str(item) for item in result] # 将结果转换为字符串列表
    http_proxy = f'http.https://github.com.proxy={proxyURL}'
    https_proxy = f'https.https://github.com.proxy={proxyURL}'
    if http_proxy in result and https_proxy in result:
        print('Git代理已成功设置。')
    else:
        print('Git代理设置失败。')

set_git_proxy(proxyURL)

## <i class="fa fa-star"></i> Notebook环境切换
在JupyterLab的Notebook中使用新的Conda环境

In [ ]:
from subprocess import getoutput
from IPython.display import clear_output
import ipywidgets as widgets

python_version = '3.10.6'

# if env_name == 'AutoDL':
custom_env_name = 'py' + python_version
# elif env_name == 'OpenBayes':
#     custom_env_name = 'base'

s = getoutput('conda info --env')
print(s)
if custom_env_name not in s:
    print(f'当前系统中没有{custom_env_name}环境，正在安装')
    # 创建Conda新的虚拟环境
    # 构建一个虚拟环境
    # 更新bashrc中的环境变量
    # 在 /openbayes/home 下创建一个新的环境：https://openbayes.com/docs/gear/conda/#%E4%BD%BF%E7%94%A8-conda-%E5%88%9B%E5%BB%BA%E6%96%B0%E7%9A%84%E7%8E%AF%E5%A2%83
    python_install_args = [
        'source /usr/local/etc/profile.d/conda.sh','&&',
        f'cd {root_dir}','&&',
        f'conda create -p {custom_env_name} python={python_version} -y','&&',
        'conda init bash','&&',
        'source /root/.bashrc -y'
    ]
    
    # !source /usr/local/etc/profile.d/conda.sh &&\
    # conda create -n $custom_env_name python=$python_version -y &&\
    # conda init bash && source /root/.bashrc -y
    
    python_install_cmd = (" ").join(python_install_args)
    !$python_install_cmd
    
else:
    print(f'已经安装了{custom_env_name}环境')

# clear_output(wait=True)
# print("Done!")

widgets.Button(
    value=False,
    description='环境部署成功!',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

## 交代Base_args

In [ ]:
from func.env import setProxy
import subprocess

def find_conda_env(name):
    # Get a list of all conda environments on the system
    output = subprocess.run(["conda", "env", "list"], capture_output=True)
    envs = output.stdout.decode().split("\n")

    # Find the path of the environment
    for env in envs:
        if name in env:
            path = env.split()[-1]
            print(f"The path of the environment '{name}' is {path}")
            return path
    else:
        print(f"Environment '{name}' not found.")
        return None



try:
    proxy,region
except NameError:
    cb=setProxy()
    proxy=cb['proxy']
    region=cb['region']
    clear_output(wait=True)

custom_env_dir = find_conda_env(custom_env_name)

base_args = [
    'source activate','&&',
    f'conda activate {custom_env_dir}','&&',
    proxy,'&&'
       ]

### 从源代码的方式安装triton

In [ ]:
# triton_repo_dir = os.path.join(root_dir,'triton')

# if not os.path.exists(triton_repo_dir):
#     !cd $root_dir &&\
#     git clone https://github.com/openai/triton.git

# !cd $root_dir &&\
# cd triton/python &&\
# pip install cmake &&\
# pip -v install -e .

### 通过PIP安装triton的最新nightly release

In [ ]:
!{" ".join(base_args + ['python -m pip install -U --pre triton'])}

## <i class="fa fa-bolt"></i> 安装Xformers和triton

大幅减少计算时间和显存占用，代价是生成的图每个都有点变化，强烈推荐安装!  
但不同核心代号的显卡需要不同的编译包，不能混用  
当然此笔记会帮你自动检测这台机器是否适合安装，而且会自动帮你安装合适的编译包

### 从源代码的方式安装Xformers

In [ ]:
import subprocess,os,time
'''
执行!python -m xformers.info命令会返回类似以下格式的信息，请参考

xFormers 0.0.16+bc08bbc.d20230122
memory_efficient_attention.cutlassF:               available
memory_efficient_attention.cutlassB:               available
memory_efficient_attention.flshattF:               available
memory_efficient_attention.flshattB:               available
memory_efficient_attention.smallkF:                available
memory_efficient_attention.smallkB:                available
memory_efficient_attention.tritonflashattF:        available
memory_efficient_attention.tritonflashattB:        available
swiglu.fused.p.cpp:                                available
is_triton_available:                               True
is_functorch_available:                            False
pytorch.version:                                   1.12.1+cu113
pytorch.cuda:                                      available
gpu.compute_capability:                            8.6
gpu.name:                                          NVIDIA GeForce RTX 3090
build.info:                                        available
build.cuda_version:                                1103
build.python_version:                              3.8.13
build.torch_version:                               1.12.1+cu113
build.env.TORCH_CUDA_ARCH_LIST:                    None
build.env.XFORMERS_BUILD_TYPE:                     None
build.env.XFORMERS_ENABLE_DEBUG_ASSERTIONS:        None
build.env.NVCC_FLAGS:                              None
build.env.XFORMERS_PACKAGE_FROM:                   None
source.privacy:                                    open source
'''

def get_python_version_and_parse():
    # Get the current python version
    version = subprocess.run(['python', '-V'], stdout=subprocess.PIPE).stdout.decode().strip()
    version = version.split(' ')[1]
    major, minor, patch = version.split('.')
    # formatted_version = f'cp{major}{minor[0]}{patch[0]}'
    formatted_version = f'cp{major}{minor[0]}'
    return formatted_version


def get_xformers_info():
    get_xformers_info_args = base_args[:]
    get_xformers_info_args += [
        # f'cd {comfyUIDir}','&&',
        # 'git pull','&&',
        # 'git checkout d8f8bcb','&&'
        'python -V',
        '&&',
        # 'conda config --append channels conda-forge','&&',
        # 'conda install --yes --file requirements.txt','&&',
        'python -m xformers.info'
    ]
    get_xformers_info_cmd = (" ").join(get_xformers_info_args)
    print(get_xformers_info_cmd)
    output = !$get_xformers_info_cmd
    output = '\n'.join(output)
    info = {}
    for line in output.strip().split('\n'):
        if ':' in line:
            key, value = line.split(':', 1)
            info[key.strip()] = value.strip()
    return info

def check_xformers_info(validate_rules):
    info = get_xformers_info()
    print((" ").join(info))
    for key, value in validate_rules.items():
        if info.get(key) != value:
            if isinstance(value, bool):
                if str(value) != info.get(key):
                    return False
            else:
                return False
    return True

import re

def find_xformers_whl_and_install(root):
    py_short_ver = get_python_version_and_parse()
    whl_name = f'xformers*{py_short_ver}*.whl'
    whl_path_list = !find $root -name $whl_name
    whl_path_list = list(filter(lambda x: x.endswith('.whl'), whl_path_list))
    if whl_path_list:
        whl_path_list = sorted(whl_path_list, key=lambda x: [int(d) if d.isdigit() else d for d in re.split(r'(\d+)', x)])
        whl_path = whl_path_list[-1]
        !{" ".join(base_args + [f'pip install {whl_path}'])}
        return whl_path
    else:
        return None

validate_rules = {
    'is_triton_available': True,
    'pytorch.cuda': 'available',
}

# 首先判断本机是否已经安装了可以运行的Xformers

if check_xformers_info(validate_rules):
    print('Xformers可以正常运行，无需另外安装或编译')
else:
    # 寻找Xformers预编译wheel，并安装返回列表中的第一个wheel
    for i in range(0, 5):  # 在input中进行搜索
        whl_path = find_xformers_whl_and_install(f'/input{i}')
        if whl_path:
            break
        else:
            continue
    

In [ ]:
# # 安装完再次验证Xformers是否能正常运作：
# if check_xformers_info(validate_rules):
#     print(f'安装后可以正常运行，无需重新编译')
# else:
#     print('需要在本机进行Xformers的编译')
#     # (Optional) Makes the build much faster
#     !pip install ninja
#     # Set TORCH_CUDA_ARCH_LIST if running and building on different GPU types
#     # !pip install -v -U git+https://github.com/facebookresearch/xformers.git@main#egg=xformers

#     xformers_repo_dir = os.path.join(root_dir,'xformers')

#     if not os.path.exists(xformers_repo_dir):
#         !cd $root_dir &&\
#         git clone https://github.com/facebookresearch/xformers.git

#     # 记录一次编译所需耗时为 6m 17.51s
#     !cd $root_dir &&\
#     cd xformers &&\
#     git show-ref main &&\
#     git symbolic-ref -q HEAD &&\
#     git submodule update --init --recursive -q &&\
#     python setup.py egg_info &&\
#     cd xformers.egg-info &&\
#     pip install -r requires.txt &&\
#     cd .. &&\
#     git rev-parse HEAD &&\
#     python setup.py bdist_wheel

#     # 安装完成后，找到对应的whl，使用pip进行安装
#     find_xformers_whl_and_install('/')

## <i class="fa fa-download"></i> 更新本体与环境

更新ComfyUI本体与环境命令

In [ ]:
comfyUIDir = os.path.join(root_dir,'ComfyUI')

In [ ]:
from IPython.display import display,clear_output
clear_output()
import ipywidgets as widgets
from func.env import setProxy

debugging=False

alertBtn = widgets.Button(
    value=False,
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='' # (FontAwesome names without the `fa-` prefix)
)


display(alertBtn)
    
if os.path.exists(comfyUIDir):
    
    alertBtn.description='正在更新...'
    alertBtn.button_style='warning'
    alertBtn.icon='upload'
    
    update_sd_webui_args = base_args[:]
    update_sd_webui_args += [
        f'cd {comfyUIDir}','&&',
        'git pull','&&',
        # 'git checkout d8f8bcb','&&'
        'python -V',
        '&&',
        # 'conda config --append channels conda-forge','&&',
        # 'conda install --yes --file requirements.txt','&&',
        'python -m pip install -r requirements.txt --user'
    ]
    update_sd_webui_cmd = (" ").join(update_sd_webui_args)
    print(update_sd_webui_cmd)
    !$update_sd_webui_cmd

    if debugging==False:
        # clear_output()
        display(alertBtn)
    
    alertBtn.description='更新完成!'
    alertBtn.button_style='success'
    alertBtn.icon='check'

else:
    alertBtn.description='安装ComfyUI...'
    alertBtn.button_style='info'
    alertBtn.icon='download'
    sd_webui_download_args = base_args[:]
    sd_webui_download_args += [
        f'cd {root_dir}','&&',
        'git clone https://github.com/comfyanonymous/ComfyUI',
        '&&',
        'cd ComfyUI','&&',
        # 'git checkout d8f8bcb','&&'
        'python -V',
        '&&',
        # 'conda config --append channels conda-forge','&&',
        # 'conda install --yes --file requirements.txt','&&',
        'python -m pip install -r requirements.txt --user'
    ]
    sd_webui_download_cmd = (" ").join(sd_webui_download_args)
    print(sd_webui_download_cmd)
    !$sd_webui_download_cmd
    
#     r = !source activate &&\
#     conda activate py3.10.6 &&\
#     $proxy &&\
#     cd $comfyUIDir &&\
#     git pull &&\
#     python -V &&\
#     python -m pip install -r requirements.txt

#     display(r)
    
#     if 'Already up to date.' in r:
#         alertBtn.description='更新完成!'
#         alertBtn.button_style='success'
#         alertBtn.icon='check'
#     else:
#         alertBtn.description='更新失败!'
#         alertBtn.button_style='danger'
#         alertBtn.icon='exclamation'


# clear_output(wait=True)

In [ ]:
!{" ".join(base_args + ['pip install transformers --upgrade --user'])}

## 交代挂载方式：复制or创建软连接

In [ ]:
%cd $repo_root_dir
%run ./nbfunc/hub.ipynb
copy_or_mount = True # True意味着只创建符号连接

# 安装自定义节点

## ControlNet预处理器节点

In [ ]:
custom_nodes_dir = os.path.join(comfyUIDir,'./custom_nodes')
ccp_dir = os.path.join(custom_nodes_dir,'./comfy_controlnet_preprocessors')

# dep_install_cmd = (' ').join(base_args[:] + [
#     f'conda install -r {custom_nodes_dir}/requirements.txt --extra-index-url https://download.pytorch.org/whl/cu117 --no-warn-script-location'
# ])
                   
# !$dep_install_cmd

mmcv_dir = os.path.join(root_dir,'./mmcv')

if not os.path.exists(mmcv_dir):
    clone_mvcc_cmd = (' ').join(base_args[:] + [
        f'cd {root_dir}','&&',
        'git clone https://github.com/open-mmlab/mmcv.git'
    ])
    !$clone_mvcc_cmd

mvcc_install_cmd = (' ').join(base_args[:] + [
    f'cd {mmcv_dir}','&&',
    'pip install -e . --user'
])
!$mvcc_install_cmd

clone_ccp_cmd = (' ').join(base_args[:]+[
    f'cd {custom_nodes_dir}','&&',
    'git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors'
])

!$clone_ccp_cmd

if os.path.exists(ccp_dir):
    # 模型
    models = [
        ('dpt_hybrid-midas-501f0c75.pt', './'),  # midas模型
        ('res101.pth', './'),  # res101.pth模型
        ('network-bsds500.pth', './'),  # hed模型
        ('mlsd_large_512_fp32.pth', './'),  # mlsd模型
        ('body_pose_model.pth', './'),  # body_pose_model模型
        ('hand_pose_model.pth', './'),  # hand_pose_model模型
        ('upernet_global_small.pth', './'),  # Segmentation的preprocessor模型
        ('table5_pidinet.pth', './pidinet')
    ]

    # 遍历模型列表并创建符号链接
    for model, directory in models:
        check_model_and_make_symbollink_general(model, os.path.join('./ckpts',directory), copy_or_mount, ccp_dir)
    
install_cmd = (' ').join(base_args[:] + [
    f'cd {ccp_dir}','&&',
    'pip uninstall opencv-python -y','&&',
    'pip install opencv-python --user','&&',
    'conda install opencv -y','&&',
    'python -V','&&',
    'pwd','&&'
    'python install.py'
])

print(install_cmd)

!$install_cmd

# 挂载各类到ComfyUI模型文件夹

## 挂载.ckpt或.safetensors

In [ ]:
import yaml

def read_yaml_file(file_path):
    with open(file_path, 'r') as f:
        yaml_data = yaml.safe_load(f)
    return yaml_data

# 调用函数并传入YAML文件路径
models_yaml_path = os.path.join(repo_root_dir,'./configs/models_list/models.yaml')
models_list = read_yaml_file(models_yaml_path)

In [ ]:
# Stable-diffusion 模型
sd_models = models_list['sd_models']

# 遍历模型列表并创建符号链接
for model in sd_models:
    check_model_and_make_symbollink_general(model, './models/checkpoints', copy_or_mount,comfyUIDir)

# VAE模型
vae_models = models_list['vae_models']

# 遍历模型列表并创建符号链接
for model in vae_models:
    check_model_and_make_symbollink_general(model, './models/vae', copy_or_mount,comfyUIDir)

# 挂载CLIP模型

## ViT-L模型

In [ ]:
import os

# torch_hub_checkpoints_dir = os.path.abspath(os.path.join(root_dir,'.torch/hub/checkpoints/'))
# print(torch_hub_checkpoints_dir)

# 该模型的下载地址为：https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth
check_model_and_make_symbollink_general('model_large_caption.pth','.torch/hub/checkpoints/',copy_or_mount,root_dir)

# Clip-Interrogator 模型
vit_models = [
    'ViT-L-14_openai_artists.safetensors',
    'ViT-L-14_openai_flavors.safetensors',
    'ViT-L-14_openai_mediums.safetensors',
    'ViT-L-14_openai_movements.safetensors',
    'ViT-L-14_openai_negative.safetensors',
    'ViT-L-14_openai_trendings.safetensors',
    # BLIP
    'model_base_caption_capfilt_large.pth',
    'ViT-L-14.pt',
    'ViT-B-16.pt',
    # Deepbooru模型
    'model-resnet_custom_v3.pt'
]

# 遍历模型列表并创建符号链接
for model in vit_models:
    check_model_and_make_symbollink_general(model, './models/clip', copy_or_mount, comfyUIDir)

## 挂载TI·Embedding

In [ ]:
# 嵌入模型
embeddings = models_list['embeddings']

# 遍历模型列表并创建符号链接
for model in embeddings:
    check_model_and_make_symbollink_general(model, './models/embeddings', copy_or_mount, comfyUIDir)

# 挂载LoRA模型

In [ ]:
import os

def create_model_with_thumbnail(model_name, models_dir, use_symlink, repo_dir):
    #先复制LoRA模型
    input_dir = check_model_and_make_symbollink_general(model_name, models_dir, use_symlink, repo_dir)
    
    if input_dir:
        file_name, ext = os.path.splitext(model_name)
        thumbnail_name = file_name + '.png'
        thumbnail_path = os.path.join(input_dir, thumbnail_name)

        thumbnail_exts = ['png', 'jpg', 'jpeg', 'bmp', 'gif']
        for ext in thumbnail_exts:
            thumbnail_name = file_name + '.' + ext
            thumbnail_path = os.path.join(input_dir, thumbnail_name)
            if os.path.exists(thumbnail_path):
                print(f'找到缩略图：{thumbnail_name}')
                # 再复制缩略图
                check_model_and_make_symbollink_general(thumbnail_name, models_dir, use_symlink, repo_dir)
                return
        print('未找到缩略图')

SD_orig_Lora_dir = './models/loras'
# Additional_Networks_Lora_dir = './extensions/sd-webui-additional-networks/models/lora'

# create_symlink(
#     source_path = os.path.abspath(os.path.join(webUIDir,SD_orig_Lora_dir)),
#     symlink_path = os.path.abspath(os.path.join(webUIDir,Additional_Networks_Lora_dir)),
# )

for i in [SD_orig_Lora_dir]:
    # 模型
    LoRA_models = models_list['LoRA_models']

    # 遍历模型列表并创建模型
    for model in LoRA_models:
        create_model_with_thumbnail(model, i, copy_or_mount,comfyUIDir)

### Upscale模型

In [ ]:
models_to_check = [
    'SwinIR_4x.pth',
    'RealESRGAN_x4plus.pth',
    'BSRGAN.pth',
    '4x_foolhardy_Remacri.pth',
    'ESRGAN_4x.pth'
]

for model in models_to_check:
    check_model_and_make_symbollink_general(model, './models/upscale_models', copy_or_mount, comfyUIDir)

## 部署ControlNet模型
解压ControlNet模型并在controlnet拓展的模型目录下创建对应模型的符号链接

### ControlNet的preprocessors模型

In [ ]:
# # 模型
# models = [
#     ('dpt_hybrid-midas-501f0c75.pt', 'midas'),  # midas模型
#     ('res101.pth', 'leres'),  # res101.pth模型
#     ('network-bsds500.pth', 'hed'),  # hed模型
#     ('mlsd_large_512_fp32.pth', 'mlsd'),  # mlsd模型
#     ('body_pose_model.pth', 'openpose'),  # body_pose_model模型
#     ('hand_pose_model.pth', 'openpose'),  # hand_pose_model模型
#     ('upernet_global_small.pth', 'uniformer')  # Segmentation的preprocessor模型
# ]

# # 遍历模型列表并创建符号链接
# for model, directory in models:
#     check_model_and_make_symbollink_general(model, f'./extensions/sd-webui-controlnet/annotator/{directory}', copy_or_mount)

### ControlNet的模型

In [ ]:
# 模型名称
controlnet_models = [
    'control_canny-fp16.safetensors',
    'control_depth-fp16.safetensors',
    'control_hed-fp16.safetensors',
    'control_mlsd-fp16.safetensors',
    'control_normal-fp16.safetensors',
    'control_openpose-fp16.safetensors',
    'control_scribble-fp16.safetensors',
    'control_seg-fp16.safetensors'
]

# 遍历模型名称列表并创建符号链接
for model_name in controlnet_models:
    check_model_and_make_symbollink_general(model_name, './models/controlnet', copy_or_mount,comfyUIDir)

### T2I模型

In [ ]:
# 模型名称
t2i_models = [ 
    ('t2iadapter_canny_sd14v1.pth', 'sketch_adapter_v14.yaml'), 
    ('t2iadapter_sketch_sd14v1.pth', 'sketch_adapter_v14.yaml'),
    ('t2iadapter_seg_sd14v1.pth', 'image_adapter_v14.yaml'),
    ('t2iadapter_keypose_sd14v1.pth', 'image_adapter_v14.yaml'),  
    ('t2iadapter_openpose_sd14v1.pth', 'image_adapter_v14.yaml'), 
    ('t2iadapter_color_sd14v1.pth', 't2iadapter_color_sd14v1.yaml'), 
    ('t2iadapter_style_sd14v1.pth', 't2iadapter_style_sd14v1.yaml')
]

def copy_and_rename_yaml(ref_yaml, model_name, target_dir):
    """
    复制并重命名给定的配置文件。
    将源文件复制到目标目录，重命名为与模型名称相同的名称（不包括扩展名）。
    Args:
        ref_yaml (str): 需要复制的配置文件的文件名。
        model_name (str): 目标模型的名称。
        target_dir (str): 目标目录的路径。
    """
    # 从 ref_yaml 中提取文件名和扩展名
    ref_name, ref_ext = os.path.splitext(ref_yaml)

    # 构建目标文件名
    target_name = os.path.splitext(model_name)[0] + ref_ext
    target_path = os.path.join(target_dir, target_name)
    
    # 检查目标路径是否存在相应的文件
    if os.path.exists(target_path):
        print(f"{target_path} 已经存在,跳过复制")
        return
    
    yaml_path = os.path.join(target_dir, ref_yaml)

    # 复制源文件并将其重命名为目标文件名
    shutil.copyfile(yaml_path, target_path)

# 遍历模型名称列表并创建符号链接
target_dir = os.path.join(comfyUIDir, './models/controlnet')

for model_name, ref_yaml in t2i_models:
    check_model_and_make_symbollink_general(model_name, target_dir, copy_or_mount, comfyUIDir)
    # copy_and_rename_yaml(ref_yaml, model_name, target_dir)

# <i class="fa fa-rocket"></i> 启动Stable-Diffusion-WebUI

### Run ComfyUI with localtunnel (Recommended Way)




In [ ]:
import shutil

if shutil.which("lt") is None:
    print("Installing localtunnel...")
    !npm install -g localtunnel
else:
    print("Localtunnel is already installed.")

import threading
import time
import socket
import os

def iframe_thread(port, subdomain):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)")
    p = subprocess.Popen(["lt", "--port", "{}".format(port), "--subdomain", "{}".format(subdomain)], stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line.decode(), end='')


port = 8188
subdomain = 'logicomfyui'

launch_args = base_args[:] + [
    f'cd {comfyUIDir}','&&',
    'python main.py',
    f'--port {port}',
    # '--dont-print-server' # 使用这个flag将会把服务端的打印静音
]

launch_cmd = (" ").join(launch_args)
print(launch_cmd)
        
threading.Thread(target=iframe_thread, daemon=True, args=(port,subdomain)).start()

!$launch_cmd
# !python main.py --dont-print-server